In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 500)

In [2]:
ddf = pd.read_csv('../scraper/datasets/disinfo.csv', index_col=0)
pdf = pd.read_csv('../scraper/datasets/polygraph.csv', index_col=0) # labels fix
sdf = pd.read_csv('../scraper/datasets/stopfake.csv', index_col=0) # translate

In [3]:
pdf['1'].unique()

array(['MISLEADING', 'LIKELY FALSE', 'FALSE', 'UNSUBSTANTIATED',
       'UNCERTAIN', 'FALSE AND MISLEADING', 'PARTLY FALSE AND MISLEADING',
       'PARTIALLY TRUE', 'UNCLEAR', 'LIKELY TRUE', 'TRUE',
       'FALSE OR MISLEADING', 'PARTLY FALSE', 'DANGEROUSLY FAKE',
       'PARTIALLY FALSE', 'MULTIPLE CLAIMS (SEE ARTICLE)', 'MOSTLY FALSE',
       'UNCLEAR AND PARTIALLY TRUE'], dtype=object)

In [4]:
pdf_newlabels = {
  'MISLEADING': 'meter-false', 'LIKELY FALSE': 'meter-false', 'FALSE': 'meter-false', 'UNSUBSTANTIATED': 'meter-false',
       'UNCERTAIN': 'meter-false', 'FALSE AND MISLEADING': 'meter-false', 'PARTLY FALSE AND MISLEADING': 'meter-false',
       'PARTIALLY TRUE': 'meter-false', 'UNCLEAR': 'meter-false', 'LIKELY TRUE': 'meter-true', 'TRUE': 'meter-true',
       'FALSE OR MISLEADING': 'meter-false', 'PARTLY FALSE': 'meter-false', 'DANGEROUSLY FAKE': 'meter-false',
       'PARTIALLY FALSE': 'meter-false', 'MULTIPLE CLAIMS (SEE ARTICLE)': 'unknown', 'MOSTLY FALSE': 'meter-false',
       'UNCLEAR AND PARTIALLY TRUE': 'meter-false'
}
pdf['truth_value'] = pdf['1']
pdf['truth_value'] = pdf['truth_value'].replace(pdf_newlabels)
pdf.head()

,0,1,truth_value
0,"Juxtaposing TikTok with Apple, Chinese Comment...",MISLEADING,meter-false
1,"Russia Unrealistically Forecasts Producing 1,5...",LIKELY FALSE,meter-false
2,Prigozhin Writes to Secretary Blinken with Fal...,FALSE,meter-false
3,Putin Falsely Claims Russia Guided by the UN C...,FALSE,meter-false
4,"Once Again, Iran’s Supreme Leader Falsely Pins...",FALSE,meter-false


In [5]:
pdf.drop([1026], inplace = True)
pdf.reset_index(drop=True)
pdf.head()

,0,1,truth_value
0,"Juxtaposing TikTok with Apple, Chinese Comment...",MISLEADING,meter-false
1,"Russia Unrealistically Forecasts Producing 1,5...",LIKELY FALSE,meter-false
2,Prigozhin Writes to Secretary Blinken with Fal...,FALSE,meter-false
3,Putin Falsely Claims Russia Guided by the UN C...,FALSE,meter-false
4,"Once Again, Iran’s Supreme Leader Falsely Pins...",FALSE,meter-false


In [6]:
col_list = list(pdf)
col_list[1], col_list[2] = col_list[2], col_list[1]
pdf.columns = col_list
pdf.head()

,0,truth_value,1
0,"Juxtaposing TikTok with Apple, Chinese Comment...",MISLEADING,meter-false
1,"Russia Unrealistically Forecasts Producing 1,5...",LIKELY FALSE,meter-false
2,Prigozhin Writes to Secretary Blinken with Fal...,FALSE,meter-false
3,Putin Falsely Claims Russia Guided by the UN C...,FALSE,meter-false
4,"Once Again, Iran’s Supreme Leader Falsely Pins...",FALSE,meter-false


In [7]:
pdf.to_csv('../scraper/datasets/polygraphv2.csv')

In [13]:
from deep_translator import GoogleTranslator

translator = GoogleTranslator(source='ru')

counter = 0
def translate(x):
  global counter
  counter += 1
  # if counter < 1610:
  #   return x
  res = translator.translate(text=x)
  print(counter, res)
  return res

sdf['claim'] = sdf['0']
sdf['claim'] = sdf['claim'].apply(translate)
sdf.head()


1 Fake: AFU soldiers fired on a car with a woman and a child for speaking Russian
2 Fake: "A concentration camp for those who do not want to fight Russia" was organized near Odessa
3 Fake: Ukrainian Recruitment Centers are heavily recruiting female chemists and biologists
4 Fake: The UN Security Council “doesn’t care about the suffering of the people of Donbass”
5 Fake: Brazilian volunteer beaten up in Ukrainian army trying to escape from position - video
6 Fake: UK transfer of uranium shells to Ukrainian Armed Forces is “nuclear escalation and new...
7 Manipulation: The UN recognized the "war crimes of the Kyiv regime in Ukraine"
8 Fake: Ukrainian refugees burned 400-year-old hotel in England
9 Manipulation: "Atheistic" Ukrainian authorities will "close access to the relics of saints" in the Kiev-Pechersk Lavra
10 Fake: Court in The Hague "acquitted" the Russian army
11 Fake: UOC-MP clergymen who came to Zelensky were kicked out with a “false” air raid alert
12 Manipulation: The Air F

,0,1,claim
0,Фейк: бойцы ВСУ обстреляли машину с женщиной и...,meter-false,Fake: AFU soldiers fired on a car with a woman...
1,Фейк: Под Одессой организовали «концлагерь для...,meter-false,"Fake: ""A concentration camp for those who do n..."
2,Фейк: Украинские центры комплектования усиленн...,meter-false,Fake: Ukrainian Recruitment Centers are heavil...
3,Фейк: Совбезу ООН «наплевать на страдания жите...,meter-false,Fake: The UN Security Council “doesn’t care ab...
4,Фейк: В украинской армии избили бразильского д...,meter-false,Fake: Brazilian volunteer beaten up in Ukraini...


In [21]:
def splitter(x, i):
  res = x.split(': ')
  if len(res) < 2:
    if i == 0:
      return 'meter-false'
    return x
  return res[i]

sdf['nc'] = sdf['claim'].apply(lambda x: splitter(x, 1))
sdf['nt'] = sdf['claim'].apply(lambda x: splitter(x, 0))
sdf.head()


,0,1,claim,nc,nt
0,Фейк: бойцы ВСУ обстреляли машину с женщиной и...,meter-false,Fake: AFU soldiers fired on a car with a woman...,AFU soldiers fired on a car with a woman and a...,Fake
1,Фейк: Под Одессой организовали «концлагерь для...,meter-false,"Fake: ""A concentration camp for those who do n...","""A concentration camp for those who do not wan...",Fake
2,Фейк: Украинские центры комплектования усиленн...,meter-false,Fake: Ukrainian Recruitment Centers are heavil...,Ukrainian Recruitment Centers are heavily recr...,Fake
3,Фейк: Совбезу ООН «наплевать на страдания жите...,meter-false,Fake: The UN Security Council “doesn’t care ab...,The UN Security Council “doesn’t care about th...,Fake
4,Фейк: В украинской армии избили бразильского д...,meter-false,Fake: Brazilian volunteer beaten up in Ukraini...,Brazilian volunteer beaten up in Ukrainian arm...,Fake


In [39]:
sdf.count()

0        1988
1        1988
claim    1988
nc       1988
nt       1988
dtype: int64

In [28]:
u = sdf['nt'].unique()

u

array(['Fake', 'Manipulation', 'meter-false', 'Fake Zakharova',
       'Video fake', 'Photofake', 'Fake Putin', '4 “atomic” fake Nebenzi',
       '"Anti-Russian conspiracy"',
       'Fake news about strikes on the port of Odessa', 'Fake Lavrova',
       'Slovak government', 'Video Fake',
       "Enemas, Moermann's diet and vitamin B17",
       '“The connection is obvious”', 'Photo manipulation',
       'Photo Manipulation', 'Seven fakes in eight minutes'], dtype=object)

In [40]:
rows_to_delete = ['Seven fakes in eight minutes', 'Photo Manipulation', '“The connection is obvious”', "Enemas, Moermann's diet and vitamin B17", '"Anti-Russian conspiracy"']
sdf = sdf[~sdf['nt'].isin(rows_to_delete)]
sdf.count()

0        1983
1        1983
claim    1983
nc       1983
nt       1983
dtype: int64

In [41]:
col_list = list(sdf)
col_list[0], col_list[3] = col_list[3], col_list[0]
sdf.columns = col_list
sdf.head()

,nc,1,claim,0,nt
0,Фейк: бойцы ВСУ обстреляли машину с женщиной и...,meter-false,Fake: AFU soldiers fired on a car with a woman...,AFU soldiers fired on a car with a woman and a...,Fake
1,Фейк: Под Одессой организовали «концлагерь для...,meter-false,"Fake: ""A concentration camp for those who do n...","""A concentration camp for those who do not wan...",Fake
2,Фейк: Украинские центры комплектования усиленн...,meter-false,Fake: Ukrainian Recruitment Centers are heavil...,Ukrainian Recruitment Centers are heavily recr...,Fake
3,Фейк: Совбезу ООН «наплевать на страдания жите...,meter-false,Fake: The UN Security Council “doesn’t care ab...,The UN Security Council “doesn’t care about th...,Fake
4,Фейк: В украинской армии избили бразильского д...,meter-false,Fake: Brazilian volunteer beaten up in Ukraini...,Brazilian volunteer beaten up in Ukrainian arm...,Fake


In [42]:
sdf.to_csv('stopfakev2.csv')

In [44]:
ddf.count()

0    2145
1    2145
dtype: int64